In [1]:
%matplotlib inline

This Notebook will take and extract all of the distances of all of the routes between each level and draw distritions of them.

In [1]:
import ipath
import os,sys
import shadow_network
from shadow_network_db_api import ShdNetworkDB
import input
from util import AccumVal

Specify here where the model files you would like to work on are and what the input file name is

In [2]:
models = {
        "Mozambique":{'modeldir':'C:\\Users\\stbrown\\Desktop\\Moz_current_2016_EPI',
                      'modelkvp':"Moz_current_2016_EPI.kvp"},
        #"Benin":{'modeldir':'D:\\Programs\\HERMES2\\oldmodels\\benin_2012',
        #         'modelkvp':'benin_input.kvp'},
        #"Bihar":{'modeldir':'D:\\Programs\\HERMES2\\oldmodels\\Bihar',
        #         'modelkvp':'Bihar.kvp'}
    }
    
os.environ["HERMES_DATA_PATH"] = "C:\\Users\stbrown\\workspaces\\HERMES2\\HERMES2\\master_data\\unified"

Create a Shadow_network model from the inputs

In [3]:
def fillDistanceDict(modeldir,modelkvp):
    cwd = os.getcwd()
    os.chdir(modeldir)
    userInput = input.UserInput(modelkvp, False)
    shdTypes = shadow_network.loadShdTypes(userInput, input.UnifiedInput())
    shdNtwk = shadow_network.loadShdNetwork(userInput, shdTypes, "model")
    
    distanceDict = {}
    for routeId,route in shdNtwk.routes.items():
        stops = route.stops
        for s in range(0,len(stops)):
            stop0 = stops[s]
            if s == len(stops)-1:
                stop1 = stops[0]
            else:
                stop1 = stops[s+1]

            stopTup = (stop0.store.CATEGORY,stop1.store.CATEGORY)
            revTup = (stop1.store.CATEGORY,stop0.store.CATEGORY)
            stopDist = stop0.DistanceKM

            if stopDist:
                if stopTup not in distanceDict.keys() and revTup not in distanceDict.keys():
                    distanceDict[stopTup] = {'distributions':{},'value':AccumVal(stopDist)}
                elif revTup in distanceDict.keys():
                    distanceDict[revTup]['value']+= stopDist
                else:
                    distanceDict[stopTup]['value'] += stopDist
    
    os.chdir(cwd)
    return shdNtwk,distanceDict

In [4]:
print os.getenv("HERMES_DATA_PATH")
modelDict = {}
for model,info in models.items():
    print info
    ntwk,distanceDict = fillDistanceDict(info['modeldir'],info['modelkvp'])
    modelDict[model] = {'shdntwk':ntwk,'distanceDict':distanceDict}
#distanceDict = fillDistanceDict(modeldir,modelkvp)

C:\Users\stbrown\workspaces\HERMES2\HERMES2\master_data\unified
{'modelkvp': 'Moz_current_2016_EPI.kvp', 'modeldir': 'C:\\Users\\stbrown\\Desktop\\Moz_current_2016_EPI'}


In [8]:
for levelTup,distAccum in distanceDict.items():
    print "Between Levels: {0} and {1}".format(levelTup[0],levelTup[1])
    print "    mean: {0}".format(distAccum.mean())
    print "    min:  {0}".format(distAccum.min())
    print "    max:  {0}".format(distAccum.max())
    print "    histogram: {0}".format(distAccum.v)

Between Levels: Provincial and Centro de Saude
    mean: 113.928571429
    min:  7.0
    max:  450.0
    histogram: [7.0, 10.0, 15.0, 18.0, 24.0, 25.0, 38.0, 40.0, 45.0, 60.0, 60.0, 60.0, 68.0, 74.0, 74.0, 75.0, 90.0, 92.0, 92.0, 112.0, 121.0, 123.0, 183.0, 215.0, 245.0, 350.0, 424.0, 450.0]
Between Levels: Centro de Saude and Centro de Saude
    mean: 41.1800362976
    min:  0.1
    max:  367.0
    histogram: [0.1, 0.1, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 9.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 11.0, 11.0, 11.0, 11.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 13.0, 13.0, 13.0, 13.0, 13.0, 14.0, 14.0, 14.0, 14.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0,

In [5]:
from scipy.stats import norm,rayleigh,exponweib
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [6]:
for model,dataDict in modelDict.items():
    ntwk = dataDict['shdntwk']
    distanceDict = dataDict['distanceDict']
    for levelTup,distD in distanceDict.items():
        # Normal Distribution Fit
        distAccum = distD['value']
        (mu,sigma) = norm.fit(distAccum.v)
        modelDict[model]['distanceDict'][levelTup]['distributions']['normal'] = (mu,sigma)
        (loc,scale) = rayleigh.fit(distAccum.v)
        modelDict[model]['distanceDict'][levelTup]['distributions']['rayleigh'] = (loc,scale)
        eFit = exponweib.fit(distAccum.v,floc=0,f0=1)
        modelDict[model]['distanceDict'][levelTup]['distributions']['wiebull'] = eFit
        n,bins,patches = plt.hist(distAccum.v,60,normed=1,facecolor='green',alpha=0.75)
        y = mlab.normpdf(bins,mu,sigma)
        y2 = rayleigh.pdf(bins,loc=loc,scale=scale)
        y3 = exponweib.pdf(bins,*eFit)

        l = plt.plot(bins,y,'r--',linewidth=2)
        l2 = plt.plot(bins,y2,'b--',linewidth=2)
        l3 = plt.plot(bins,y3,'r--',linewidth=2)
        plt.xlabel('Distances')
        plt.ylabel('Frequency')
        plt.title('{2}: Distances between The {0} and {1} Levels'.format(levelTup[0],levelTup[1],model))
        #plt.grid(true)
        plt.show()
        print "Normal Distribution: mu={0} sigma={1}".format(mu,sigma)
        print "Rayleigh Distribution: loc={0} scale={1}".format(loc,scale)
        print "Weibull Distribution: params={0}".format(eFit)

Normal Distribution: mu=113.928571429 sigma=117.624380542
Rayleigh Distribution: loc=-58.2216524717 scale=147.429985806
Weibull Distribution: params=(1, 1.0440763873381183, 0, 116.1054144624305)
Normal Distribution: mu=41.1800362976 sigma=38.6769101197
Rayleigh Distribution: loc=-14.0539518455 scale=47.6796187203
Weibull Distribution: params=(1, 1.2376271584047953, 0, 44.38418212367246)
Normal Distribution: mu=82.1157894737 sigma=60.0224279412
Rayleigh Distribution: loc=4.37467020594 scale=69.4491544828
Weibull Distribution: params=(1, 1.6101541344627783, 0, 92.985218028956936)
Normal Distribution: mu=30.6 sigma=29.6620970263
Rayleigh Distribution: loc=-12.3520340695 scale=36.9101290086
Weibull Distribution: params=(1, 1.1687383128866644, 0, 32.572895370389887)
Normal Distribution: mu=48.7851809955 sigma=43.8198442602
Rayleigh Distribution: loc=-17.2818876213 scale=56.0581809723
Weibull Distribution: params=(1, 1.1761663759300938, 0, 51.613175166302526)
Normal Distribution: mu=942.6 si

Create a large number of models with distances chosen from distributions

In [16]:
def createRandomShdNtwk(shdNtwk,distanceDict,numberOfModels=1,distType="normal"):
    from copy import copy
    import numpy as np
    modelsList = []
    for i in range(0,numberOfModels):
        newShdNtwk = shdNtwk.copy()
        for routeId,route in newShdNtwk.routes.items():
            if route.Type == "attached": continue
            ### get the right distribution
            distDict = None
            stops = route.stops
            if len(stops)==2:
                stopTup = (stops[0].store.CATEGORY,stops[1].store.CATEGORY)
                revTup = (stops[1].store.CATEGORY,stops[0].store.CATEGORY)
                if stopTup in distanceDict.keys(): distDict = distanceDict[stopTup]
                elif revTup in distanceDict.keys(): distDict = distanceDict[revTup]
                else:
                    raise RuntimeError("{0} or {1} not found in distanceDict".format(stopTup,revTup))
                
                currentSpeed = stops[0].DistanceKM/stops[0].TransitHours
                randomDistance = -1.0
                #print distDict.keys()
                if distType == "normal":
                    mu = distDict['distributions']['normal'][0]
                    sigma = distDict['distributions']['normal'][1]
                    randomDistance = np.random.normal(mu,sigma,1)
                
                
                newShdNtwk.routes[routeId].stops[0].DistanceKM = randomDistance[0]
                newShdNtwk.routes[routeId].stops[0].TransitHours = randomDistance[0]/currentSpeed
                newShdNtwk.routes[routeId].stops[1].DistanceKM = randomDistance[0]
                newShdNtwk.routes[routeId].stops[1].TransitHours = randomDistance[0]/currentSpeed
        else:
            for s in range(0,len(stops)):
                stop0 = stops[s]
                if s == len(stops)-1:
                    stop1 = stops[0]
                else:
                    stop1 = stops[s+1]

                stopTup = (stop0.store.CATEGORY,stop1.store.CATEGORY)
                revTup = (stop1.store.CATEGORY,stop0.store.CATEGORY)
                if stopTup in distanceDict.keys(): distDict = distanceDict[stopTup]
                elif revTup in distanceDict.keys(): distDict = distanceDict[revTup]
                else:
                    raise RuntimeError("{0} or {1} not found in distanceDict".format(stopTup,revTup))
                
                stopDist = stop0.DistanceKM
        modelsList.append(newShdNtwk)
    
    return modelsList

In [19]:
mList = createRandomShdNtwk(modelDict['Mozambique']['shdntwk'],modelDict['Mozambique']['distanceDict'])

In [18]:
model = mList[0]
model.writeCSVRepresentation()

****** Begin Warning ******
No file to play the role of gapstorefile in exported model_copy
****** End Warning ********
****** Begin Warning ******
No file to play the role of catchupfile in exported model_copy
****** End Warning ********
looked in ..\..\src\version.txt


In [15]:
os.getcwd()

'C:\\Users\\stbrown\\Desktop\\Moz_current_2016_EPI'